# Who's the biggest tax evader?

#### Imports:

In [ ]:
import plotly.plotly as py
import pandas as pd
import pycountry
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

## 1. Data cleaning and preprocessing

In [ ]:
# Load country codes
df_country_codes = pd.read_csv('data/countries_codes.csv', low_memory=False).set_index('COUNTRY')

In [ ]:
# Load datasets
## Load panama papers datasets
pp_edges = pd.read_csv('data/panama_papers/panama_papers.edges.csv', low_memory=False)
pp_nodes_address = pd.read_csv('data/panama_papers/panama_papers.nodes.address.csv', low_memory=False)
pp_nodes_entity = pd.read_csv('data/panama_papers/panama_papers.nodes.entity.csv', low_memory=False)
pp_nodes_intermediary = pd.read_csv('data/panama_papers/panama_papers.nodes.intermediary.csv', low_memory=False)
pp_nodes_officer = pd.read_csv('data/panama_papers/panama_papers.nodes.officer.csv', low_memory=False)
## Load UN datasets
un_hdi_components_2014 = pd.read_csv('data/un/hdi_components.csv', low_memory=False)
un_gdp_per_capita = pd.read_csv('data/un/gdp_per_capita.csv', low_memory=False)
un_gdp_per_capita_ppp = pd.read_csv('data/un/gdp_per_capita_PPP.csv', low_memory=False)
## Load world bank datasets
wb_gini = pd.read_csv('data/world_bank/gini_index.csv', low_memory=False)
wb_income_share_20_per = pd.read_csv('data/world_bank/income_share_20_per.csv', low_memory=False)
wb_population_total = pd.read_csv('data/world_bank/population_total.csv', low_memory=False)
# TIM
wb_co2 = pd.read_excel('data/co2_emissions.xls')

In [ ]:
# We only consider statistics that date from 2000 onwards
years_to_drop = list(map(str, np.arange(1960, 2000)))
wb_gini = wb_gini.drop(columns=years_to_drop)
wb_income_share_20_per = wb_income_share_20_per.drop(columns=years_to_drop)

In [ ]:
# We select the rightmost value (most recent) for each row
gini_values = wb_gini.stack().groupby(level=0).last().reindex(wb_gini.index)

# Only select valid values and label other values as NaN
wb_gini['Gini'] = pd.to_numeric(gini_values, errors='coerce')

# Only select relevant columns
wb_gini = wb_gini[['Country Name', 'Country Code', 'Gini']]

In [ ]:
# We select the rightmost value (most recent) for each row
income_share_20_per_values = wb_income_share_20_per.stack().groupby(level=0).last().reindex(wb_income_share_20_per.index)

# Only select valid values and label other values as NaN
wb_income_share_20_per['Income Share'] = pd.to_numeric(income_share_20_per_values, errors='coerce')

# Only select relevant columns
wb_income_share_20_per = wb_income_share_20_per[['Country Name', 'Country Code', 'Income Share']]

In [ ]:
un_hdi_components_2014 = un_hdi_components_2014[un_hdi_components_2014['Human Development Index (HDI)'] != '..']
un_hdi_components_2014['Human Development Index (HDI)'] = un_hdi_components_2014['Human Development Index (HDI)'].astype('float')

In [ ]:
# Join UN datasets with country codes DataFrame
un_hdi_components_2014 = un_hdi_components_2014.join(df_country_codes, on='Country')
un_gdp_per_capita = un_gdp_per_capita.join(df_country_codes, on='Country')
un_gdp_per_capita_ppp = un_gdp_per_capita_ppp.join(df_country_codes, on='Country')

In [ ]:
# List of UN DataFrames
un_dfs = [un_hdi_components_2014, un_gdp_per_capita, un_gdp_per_capita_ppp]

# Define dictionary containing pairs (country name: ISO country code)
countries = dict()

for country in pycountry.countries:
    countries[country.name] = country.alpha_3  

for df in un_dfs:
    nan_values = df['CODE'].isna()
    input_countries = list(df[nan_values]['Country'].values)
        
    codes = []
    for country in input_countries:
        if country in countries:
            codes.append(countries.get(country))
        else:        
            accepted = []
            str_country = str(country)
            # check if string contains either common or official country name
            for p_country in pycountry.countries:
                if p_country.name in str_country or (hasattr(p_country, 'common_name') and p_country.common_name in str_country):
                    accepted.append(p_country.alpha_3)
            if len(accepted) == 1:
                codes.append(accepted[0])
            else:
                codes.append(None)

    df.loc[nan_values, 'CODE'] = codes
    # Remove rows that were not found
    df = df[df['CODE'].notnull()]

In [ ]:
pp_references_country = pp_nodes_address.groupby(['country_codes', 'countries']).size().reset_index(name='counts')

In [ ]:
wb_population_2014 = wb_population_total[['Country Code', '2014']]
occurrence_pop = pp_references_country.merge(wb_population_2014, left_on='country_codes', right_on='Country Code')
occurrence_pop['counts_1000'] = 1000 * occurrence_pop['counts'] / occurrence_pop['2014']

In [ ]:
# TIM

# We select the rightmost value (most recent) for each row
wb_co2_values = wb_co2.stack().groupby(level=0).last().reindex(wb_co2.index)

# Only select valid values and label other values as NaN
wb_co2['CO2 Emissions'] = pd.to_numeric(wb_co2_values, errors='coerce')

# Only keep most recent values for each country
wb_co2 = wb_co2[['Country Name', 'Country Code', 'CO2 Emissions']]

# Remove countries without indicator information
wb_co2 = wb_co2.dropna()

## 2. Data analysis and observations

### 2.1 Panama Papers and population

In [ ]:
occurrence_pop['counts_1000'].plot.hist(title='Distribution of references per 1000 inhabitants', bins=100, logy=True)
plt.xlabel('Number of occurrences in Panama Papers per 1000 inhabitants')
plt.show()

In [ ]:
pp_intermediary_country = pp_nodes_intermediary.groupby(['country_codes', 'countries']).size().reset_index(name='counts')
pp_intermediary_country = pp_intermediary_country.sort_values('counts', ascending=False)

We display the distribution using a map:

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = pp_intermediary_country['country_codes'],
        z = pp_intermediary_country['counts'],
        text = pp_intermediary_country['countries'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = .3
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Number of references'),
      ) ]

"""
layout = {
  "geo": {
    "coastlinewidth": 2, 
    "countrycolor": "rgb(204, 204, 204)", 
    "lakecolor": "rgb(255, 255, 255)", 
    "landcolor": "rgb(204, 204, 204)", 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 60], 
      "showgrid": True
    }, 
    "lonaxis": {
      "dtick": 20, 
      "range": [-100, 20], 
      "showgrid": True
    }, 
    "projection": {"type": "equirectangular"}, 
    "resolution": 50, 
    "showlakes": True, 
    "showland": False
  }, 
  "showlegend": False, 
  "title": "Seoul to Hong Kong Great Circle"
}

"""
layout = dict(
    title = 'References in Panama Papers',
    geo = dict(
        showcountries = True,
        countrycolor = "rgb(217, 217, 217)",
        showframe = False,
        resolution=110,
        showcoastlines = False,
        bgcolor = 'rgba(255, 255, 255, 0.0)',
    )
)


fig = dict( data=data, layout=layout )

iplot( fig, validate=False)

In [ ]:
min_count = pp_intermediary_country['counts'].min()
max_count = pp_intermediary_country['counts'].max()

In [ ]:
def firstOrDefault(values, default):
    if values is None or len(values) == 0:
        return default
    return values[0]

## 3. Milestone 3

We can see from the tables that most of the countries involved in the Panama Papers affair are small islands, which unfortunately are not displayed by the `Plotly` library. For the next milestone, we will fix that issue either by finding a solution that still works with `Plotly` or by using a different library, such as `folium`.

So far, we have made insightful observations that match the reports found in the media, particularly about which countries were most involved in this affair.

For the next milestone, we will further investigate the links between the countries, and try to understand the correlation of socio-economic factors with the locations of entities, officers and intermediaries involved in Panama Papers. More specifically, we intend to:
- Find which socio-economic factors are correlated with the results we found so far, and how they are correlated
- Display the links between the countries using a graph similar to the one found [here](https://plot.ly/python/lines-on-maps/)
- Fix issues with certain countries (particularly small islands) not being displayed in the graph
-

In [ ]:
pp_edges_parsed = pp_edges[['START_ID', 'TYPE', 'END_ID']]

In [ ]:
pp_nodes_intermediary_parsed = pp_nodes_intermediary[['node_id', 'country_codes', 'countries']]
pp_nodes_entity_parsed = pp_nodes_entity[['node_id', 'country_codes', 'countries']]
pp_nodes_officer_parsed = pp_nodes_officer[['node_id', 'country_codes', 'countries']]

In [ ]:
pp_nodes = pp_nodes_entity_parsed.append(pp_nodes_intermediary_parsed).append(pp_nodes_officer_parsed)
pp_nodes = pp_nodes.dropna()

In [ ]:
pp_edges_countries = pp_nodes.merge(pp_edges_parsed, left_on='node_id', right_on='START_ID')
pp_edges_countries = pp_edges_countries.rename(columns={'node_id': 'id_origin', 'country_codes': 'cc_origin', 
                                                        'countries': 'c_origin'})
pp_edges_countries = pp_edges_countries.merge(pp_nodes, left_on='END_ID', right_on='node_id')
pp_edges_countries = pp_edges_countries.rename(columns={'node_id': 'id_dest', 'country_codes': 'cc_dest', 
                                                        'countries': 'c_dest'})
pp_edges_countries = pp_edges_countries.drop(columns=['id_origin', 'id_dest'])

## Scatter plot 

Now we will try to find a correlation between the amount of involvement in Panama Papers of the three different types of node and different social economic factors. We considered several different socio-economic factors, but ultimately we decided to focus on:
- Human Development Index
- GDP per capita
- Gini coefficient (measures inequality)
- Income share of the 20% richest

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

We define a simple function to count the number of occurrences of the various entity in the Panama Papers

In [ ]:
def count_occurence(pp_data):
    return pp_data.groupby(['country_codes', 'countries']).size().reset_index(name='counts')

We will now use this function to count the number of occurrences of each country in the scandal

In [ ]:
pp_references_officier = count_occurence(pp_nodes_officer)
pp_references_officier.sort_values('counts', ascending=False).head(10)

In [ ]:
pp_references_entity = count_occurence(pp_nodes_entity)
pp_references_entity.sort_values('counts', ascending=False).head(10)

In [ ]:
pp_references_intermediary = count_occurence(pp_nodes_intermediary)
pp_references_intermediary.sort_values('counts', ascending=False).head(9)

As we can see, most of the countries with high presence in the panama papers are either countries known to have non strict fiscal laws either country with a very high population. We will later normalize by the population of each country to delete the bias that comes from the population size.

We might be also interested by the total number of occurrences in the Panama papers regardless of the the type of node. Let's compute it:

In [ ]:
pp_references_total = pp_references_intermediary.merge(pp_references_entity, on='country_codes')
pp_references_total = pp_references_total.merge(pp_references_officier, on='country_codes')
pp_references_total['counts'] = pp_references_total['counts'] + pp_references_total['counts_x'] + pp_references_total['counts_y']
pp_references_total = pp_references_total[['country_codes', 'countries', 'counts']]


In [ ]:
pp_references_total.sort_values('counts', ascending=False).head()

We now define the different datasets in which we will try to find a correlation with the social factor 

In [ ]:
nodes_types = [pp_references_entity, pp_references_intermediary, pp_references_officier, pp_references_total]
nodes_types_desc = ['Entity references', 'Intermediary references', 'Officier references', 'Total references']


As stated above, the number of occurrences is biased by countries that have a high population. To remove this bias, we want to normalize the counts of occurrences by the population of each country.
The function that we define bellow simply does that.

In [ ]:
def normalize_count(df):
    merged = df.merge(wb_population_2014, left_on='country_codes', right_on='Country Code')[['country_codes', 'countries', 'counts', '2014']]
    merged['counts_normalized'] = merged['counts'] / merged['2014']
    return merged

In [ ]:
# Normalizing every dataset
nodes_types_normalized = list(map(normalize_count, nodes_types))

### GDP
The GDP per capita is the value of all the goods and services produced by a country in one year. It therefore represents approximately how rich a country is. We chose to use this indicator because we thought that richer people might have more incentive to try to evade the tax system. We will go deeper in this analyze later. 

First, we start by selecting only the GDP per capita of 2015 (year of the affair). We will then merge the GDP dataset with all the datasets of the various type of nodes.

In [ ]:
gdp_2015 = un_gdp_per_capita[un_gdp_per_capita['Year'] == 2015][['CODE', 'Value']]
gdp_data = [x.merge(gdp_2015, left_on='country_codes', right_on='CODE') for x in nodes_types_normalized]


We now define a function that will create a scatter plot between an economic index and all the types of nodes. This function will also compute the Pearson and Spearman correlation coefficient and give us the p-value.

In [ ]:
def plot_all_entity(data, x_name, y_name, data_name, index_activated, title, x_label, y_label, save_name):
    traces = []
    i = 0
    for d in data:
        # Create a trace
        trace = go.Scatter(
            name = data_name[i],
            x = d[x_name],
            y = d[y_name],
            mode = 'markers',
            text = d['countries'],
            visible = 'legendonly' if i != index_activated else True
        )
        traces.append(trace)
        
        corr, p = stats.pearsonr(d[x_name], d[y_name])
        print(data_name[i])
        print('\tpearson : ', corr, '; p-val: ', p)
        corr, p = stats.spearmanr(d[x_name], d[y_name])
        print('\tspearman : ', corr, '; p-val: ', p)
        i+=1
    data = traces
    
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title=x_label
        ),
        yaxis=dict(
            title=y_label
        ),
        hovermode = 'closest'
    )
    fig = go.Figure(data=data, layout=layout)
    file_path = '../TrovatelliT.github.io/ressources/' + save_name
    iplot(fig)
    plot(fig, filename=file_path, auto_open=False) # Also save the plot for the website


We display the scatter plot showing the correlation between the GDP per capita and the number of total occurrences normalized in the scandal. As said earlier, we will only consider the number of occurrences normalized since countries with high populations will render the number of counts more biased.

In [ ]:
plot_all_entity(gdp_data, 'counts_normalized', 'Value', nodes_types_desc, 3, 'GDP per capita vs number of occurrences in the scandal', 'Number of occurrences in the Panama Papers per inhabitant', 'GDP per capita', 'scatter_gdp_count.html')


As we can see most there are a few outliers. Most of them are fiscal paradise that have a high number of occurrences and a low number of population (British Virgin Islands, Luxembourg, Bermuda, etc.). As we're plotting the number of occurrences normalized, we would expect such countries to stand out.

To solve this problem we'll only keep points $x$ that respect these conditions:

$$q_{.25} - 1.5 \cdot \text{IQR} \leq x \leq q_{.75} + 1.5 \cdot \text{IQR}$$

where : 

- $\text{IQR}$ is the interquartile range defined by: $q_{.75} - q_{.25}$
- $q_t$ is the t-quantile of this distribution

From now on, we'll always remove the outliers of every plots

We'll also compute the p-value.
Our null hypothesis $H_0$ is that the two variables are uncorrelated. We can reject $H_0$ when the p-value is high enough. We'll fix our threshold $\alpha = .05$. So if a p-value is higher than $.05$ we can reject the fact that the two variables are uncorrelated.

Below we'll only analyze variables that have a significant correlation.


The function below implement that.

In [ ]:
def remove_outliers(data, field1, field2):
    Q1_1 = data[field1].quantile(0.25)
    Q3_1 = data[field1].quantile(0.75)
    IQR_1 = Q3_1 - Q1_1
    
    Q1_2 = data[field2].quantile(0.25)
    Q3_2 = data[field2].quantile(0.75)
    IQR_2 = Q3_2 - Q1_2

    filter1 = (data[field1] < Q3_1 + 1.5 * IQR_1) & (data[field1] > Q1_1 - 1.5 * IQR_1)
    filter2 = (data[field2] < Q3_2 + 1.5 * IQR_2) & (data[field2] > Q1_2 - 1.5 * IQR_2)
    return data[filter1 & filter2]

We'll now plot the same plot without the outliers.

In [ ]:
data = list(map(lambda d: remove_outliers(d, 'Value', 'counts_normalized'), gdp_data))
plot_all_entity(data, 'counts_normalized', 'Value', nodes_types_desc, 3, 'GDP per capita vs number of occurrences in the scandal without outliers', 'Number of occurrences in the Panama Papers per inhabitant', 'GDP per capita', 'scatter_gdp_count_outlier.html')


Even if on this graph the correlation are smaller, it is can better see the pattern that the number are suggesting. 
We can see that for the officers the correlation coefficients are very high. The p-values are very small and allow us to reject the fact that the references of officer and the GDP is uncorrelated.


We observe that countries where the GDP is higher appear more often in the Panama Papers. Again, this is what we would intuitively predict.
Richer countries have usually better infrastructure and therefore have a system to ensure that the taxes are actually collected.
This provides a much larger incentive for people to try to evade this systems by any means they can, which leads to scandals such as this one.

### Gini Index

The Gini coefficient measures the inequality in the distribution of wealth in a country (lower coefficient means lower inequality). The most equal equal society is when every person get the same income (When gini = 0). Our initial assumption was that countries in which there is high levels of income inequality would be more heavily involved in this affair, as this is generally heavily linked with corruption and tax evasion.

In [ ]:
gini_nan = wb_gini.dropna()

We do the same as for the GDP. We join the dataset with the Gini coefficients and with the number of occurrences in the scandal. We then only show the plot without the outliers.

In [ ]:
gini_data = [x[x['counts'] > 0].merge(gini_nan, left_on='country_codes', right_on='Country Code') for x in nodes_types_normalized]

In [ ]:
data = list(map(lambda d: remove_outliers(d, 'Gini', 'counts_normalized'), gini_data))
plot_all_entity(data, 'counts_normalized', 'Gini', nodes_types_desc, 3, 'Gini coefficient vs number of occurrenes in the scandal without outliers', 'Number of occurrences per inhabitant', 'Gini Index', 'scatter_gini_count_outlier.html')

As we can see in the table above, the correlation between the number of occurrence and the Gini coefficient is not really relevant as the high p-value suggests.

We can then not reject the hypothesis $H_0$ that the two variables are uncorrelated.

Which is surprising !
Our initial assumption was that countries in which there is high levels of income inequality would be more heavily involved in this affair. As the tax evasion are higher in those countries, the redistribution of wealth is reduced.



### Income held by top 20%

This index also measures inequality but it quantifies it differently by expressing the share of wealth held to the top 20% richest. We would expect similar results as for the Gini coefficient since both indices are measuring inequality.

We'll again show the plot without the outliers.

In [ ]:
wb_income_share_nan = wb_income_share_20_per.dropna()

In [ ]:
income_share_data = [x.merge(wb_income_share_nan, left_on='country_codes', right_on='Country Code') for x in nodes_types_normalized]


In [ ]:
data = list(map(lambda d: remove_outliers(d, 'counts_normalized', 'Income Share'), income_share_data))
plot_all_entity(data, 'counts_normalized', 'Income Share', nodes_types_desc, 3, 'Wealth held by 20% richest vs number of occurrences in the scandal', 'Number of occurences per inhabitant', 'Wealth held by 20% richest [%]', 'scatter_20_count_outliers.html')


All the correlation coefficients are low and the p-values are very high. We can again not reject $H_0$.
There is almost no correlation which is coherent with what we observed before for the Gini coefficient.

Unfortunately, we do not get anymore insight by using this metric when studying the involvement of a country in Panama Papers.


### HDI

The HDI index try to represent how well a country is developed by using the lifespan, the education level and the GDP per capita of an average citizen to compute the index.  
We now observe if there is a correlation between the HDI and the number of occurrence in this affair.

As we've done so far, we will show the scatter plot of the HDI vs the number of occurrences in the Panama Papers without the outliers.

In [ ]:
hdi_data = [x.merge(un_hdi_components_2014, left_on='country_codes', right_on='CODE') for x in nodes_types_normalized]


In [ ]:
data = list(map(lambda d: remove_outliers(d, 'Human Development Index (HDI)', 'counts_normalized'), hdi_data))
plot_all_entity(data, 'counts_normalized', 'Human Development Index (HDI)', nodes_types_desc, 3, 'HDI vs number of occurrences in the scandal', 'Number of occurences per inhabitant', 'HDI Index', 'scatter_hdi_count_outlier.html')


The p-value suggest that we can reject $H_0$, therefore the variables are correlated. This is map were we can best visually see the correlation. It seems that for an $\text{HDI} \geq 0.65$ there is a big increase in the number of occurrences in the Panama Papers.

There seems to be a correlation as the plot and the Spearman coefficient suggest.
One of the reasons could be that a country with higher HDI have generally a better infrastructures to ensure that the tax are paid. Additionally, a country with higher HDI is home to richer people since the GDP per capita is also taken into account in the calculation of the HDI. Or maybe it is the case because richer people like to live in developed country? 

In this section we studied the impact of socioeconomic factor in the way countries are involved in the Panama papers. We saw that we couldn't show that the degree of inequality within a country influenced how tied to this affair a country was. However we saw that there is a medium a correlation with the GDP per capita and with the HDI as well.